简答题：
1. 如果你的训练集具有数百万个特征，那么可以使用哪种线性回归训练算法？

2. 如果你的训练集里特征的数值大小迥异，那么哪些算法可能会受到影响？受影响程度如何？你应该怎么做？

3. 训练逻辑回归模型时，梯度下降可能会卡在局部最小值中吗？

4. 如果你让它们运行足够长的时间，是否所有的梯度下降算法都能得出相同的模型？

5. 假设你使用批量梯度下降，并在每个轮次绘制验证误差。如果你发现验证错误持续上升，那么可能是什么情况？你该如何解决？

6. 当验证误差上升时立即停止小批量梯度下降是个好主意吗？

7. 哪种梯度下降算法（在我们讨论过的算法中）将最快地到达最佳解附近？哪个实际上会收敛？如何使其他的也收敛

8. 假设你正在使用多项式回归。绘制学习曲线后，你会发现训练误差和验证误差之间存在很大的差距。发生了什么？解决此问题的三种方法是什么？

9. 假设你正在使用岭回归，并且你注意到训练误差和验证误差几乎相等且相当高。你是否会说模型存在高偏差或高方差？你应该增加正则化超参数α还是减小它呢？

10. 为什么要使用：a.岭回归而不是简单的线性回归（即没有任何正则化）？b.Lasso而不是岭回归？c.弹性网络而不是Lasso回归？

11. 假设你要将图片分类为室外/室内和白天/夜间。你应该实现两个逻辑回归分类器还是一个softmax回归分类器？

编程题：

In [183]:
# todo 编程题: 在不使用sklearn的情况下，仅使用Numpy，为softmax回归实现带早停的批量梯度下降，将它用于分类任务，
#  例如鸢尾花数据集  load_iris, 只用两个特征就可以："petal width (cm)", "petal length (cm)"
#  强调：除了读数据，其他全用numpy （包括分离测试+验证），不用sklearn

#  注意：
#  1. 要实现l2正则化
#  2. 除了数据读取，其他仅使用numpy，包括训练集+验证集分离，以及softmax预测 和 损失计算

In [1]:
import numpy as np
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt

iris = load_iris()
X = iris.data[:, [2, 3]]
y = iris.target
X,y

(array([[1.4, 0.2],
        [1.4, 0.2],
        [1.3, 0.2],
        [1.5, 0.2],
        [1.4, 0.2],
        [1.7, 0.4],
        [1.4, 0.3],
        [1.5, 0.2],
        [1.4, 0.2],
        [1.5, 0.1],
        [1.5, 0.2],
        [1.6, 0.2],
        [1.4, 0.1],
        [1.1, 0.1],
        [1.2, 0.2],
        [1.5, 0.4],
        [1.3, 0.4],
        [1.4, 0.3],
        [1.7, 0.3],
        [1.5, 0.3],
        [1.7, 0.2],
        [1.5, 0.4],
        [1. , 0.2],
        [1.7, 0.5],
        [1.9, 0.2],
        [1.6, 0.2],
        [1.6, 0.4],
        [1.5, 0.2],
        [1.4, 0.2],
        [1.6, 0.2],
        [1.6, 0.2],
        [1.5, 0.4],
        [1.5, 0.1],
        [1.4, 0.2],
        [1.5, 0.2],
        [1.2, 0.2],
        [1.3, 0.2],
        [1.4, 0.1],
        [1.3, 0.2],
        [1.5, 0.2],
        [1.3, 0.3],
        [1.3, 0.3],
        [1.3, 0.2],
        [1.6, 0.6],
        [1.9, 0.4],
        [1.4, 0.3],
        [1.6, 0.2],
        [1.4, 0.2],
        [1.5, 0.2],
        [1.4, 0.2],


In [2]:
def standardize(X):
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    return (X - mean) / std

X = standardize(X)
X

array([[-1.34022653e+00, -1.31544430e+00],
       [-1.34022653e+00, -1.31544430e+00],
       [-1.39706395e+00, -1.31544430e+00],
       [-1.28338910e+00, -1.31544430e+00],
       [-1.34022653e+00, -1.31544430e+00],
       [-1.16971425e+00, -1.05217993e+00],
       [-1.34022653e+00, -1.18381211e+00],
       [-1.28338910e+00, -1.31544430e+00],
       [-1.34022653e+00, -1.31544430e+00],
       [-1.28338910e+00, -1.44707648e+00],
       [-1.28338910e+00, -1.31544430e+00],
       [-1.22655167e+00, -1.31544430e+00],
       [-1.34022653e+00, -1.44707648e+00],
       [-1.51073881e+00, -1.44707648e+00],
       [-1.45390138e+00, -1.31544430e+00],
       [-1.28338910e+00, -1.05217993e+00],
       [-1.39706395e+00, -1.05217993e+00],
       [-1.34022653e+00, -1.18381211e+00],
       [-1.16971425e+00, -1.18381211e+00],
       [-1.28338910e+00, -1.18381211e+00],
       [-1.16971425e+00, -1.31544430e+00],
       [-1.28338910e+00, -1.05217993e+00],
       [-1.56757623e+00, -1.31544430e+00],
       [-1.

In [4]:
def train_val_split(X, y, val_size=0.2, random_state=42):
    if random_state is not None:
        np.random.seed(random_state)

    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)

    split_idx = int(X.shape[0] * (1 - val_size))

    X_train, X_val = X[indices[:split_idx]], X[indices[split_idx:]]
    y_train, y_val = y[indices[:split_idx]], y[indices[split_idx:]]

    return X_train, X_val, y_train, y_val

X_train, X_test, y_train, y_test = train_val_split(X, y, val_size=0.2, random_state=42)
X_test,y_test

(array([[ 4.78571135e-01,  2.64141916e-01],
        [-1.39706395e+00, -1.18381211e+00],
        [ 4.78571135e-01,  1.32509732e-01],
        [ 3.64896281e-01,  8.77547895e-04],
        [-1.28338910e+00, -1.31544430e+00],
        [ 1.94384000e-01,  1.32509732e-01],
        [ 1.44480739e+00,  7.90670654e-01],
        [ 1.10378283e+00,  1.18556721e+00],
        [-1.28338910e+00, -1.05217993e+00],
        [-2.60315415e-01, -2.62386821e-01],
        [ 1.10378283e+00,  1.71209594e+00],
        [ 1.16062026e+00,  5.27406285e-01],
        [-1.34022653e+00, -1.44707648e+00],
        [ 1.04694540e+00,  1.58046376e+00],
        [-1.34022653e+00, -1.31544430e+00],
        [ 6.49083415e-01,  3.95774101e-01],
        [ 1.33113254e+00,  9.22302838e-01],
        [ 1.04694540e+00,  7.90670654e-01],
        [ 1.94384000e-01,  1.32509732e-01],
        [ 9.90107977e-01,  7.90670654e-01],
        [ 3.64896281e-01,  1.32509732e-01],
        [ 3.08058854e-01,  1.32509732e-01],
        [ 6.49083415e-01,  1.053

In [5]:
X_train = np.hstack([np.ones((X_train.shape[0], 1)), X_train])
X_test = np.hstack([np.ones((X_test.shape[0], 1)), X_test])

In [6]:
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

In [8]:
def compute_loss(X, y, weights, lambda_reg):
    m = X.shape[0]
    n_classes = weights.shape[1]

    y_pred = softmax(np.dot(X, weights))
    cross_entropy = -np.mean(np.log(y_pred[np.arange(m), y] + 1e-10))
    l2_loss = 0.5 * lambda_reg * np.sum(weights[1:] ** 2) / m
    return cross_entropy + l2_loss

In [9]:
def compute_gradient(X, y, weights, lambda_reg):
    m = X.shape[0]
    n_classes = weights.shape[1]

    y_pred = softmax(np.dot(X, weights))

    y_one_hot = np.zeros((m, n_classes))
    y_one_hot[np.arange(m), y] = 1

    gradient = np.dot(X.T, (y_pred - y_one_hot)) / m
    gradient[1:] += lambda_reg * weights[1:] / m

    return gradient

In [11]:
def batch_gradient_descent(X_train, y_train, X_test, y_test,
                          learning_rate=0.1, lambda_reg=0.01,
                          epochs=1000, batch_size=32, patience=10):
    n_features = X_train.shape[1]
    n_classes = len(np.unique(y_train))

    weights = np.random.randn(n_features, n_classes) * 0.01

    best_val_loss = float('inf')
    best_weights = None
    counter = 0

    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        indices = np.arange(X_train.shape[0])
        np.random.shuffle(indices)
        X_shuffled = X_train[indices]
        y_shuffled = y_train[indices]

        for i in range(0, X_train.shape[0], batch_size):
            X_batch = X_shuffled[i:i+batch_size]
            y_batch = y_shuffled[i:i+batch_size]

            gradient = compute_gradient(X_batch, y_batch, weights, lambda_reg)
            weights -= learning_rate * gradient

        train_loss = compute_loss(X_train, y_train, weights, lambda_reg)
        val_loss = compute_loss(X_test, y_test, weights, lambda_reg)

        train_losses.append(train_loss)
        val_losses.append(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_weights = weights.copy()
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f"早停在第 {epoch+1} 轮")
                break

    return best_weights, train_losses, val_losses

In [12]:
def predict(X, weights):
    y_pred = softmax(np.dot(X, weights))
    return np.argmax(y_pred, axis=1)

def accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

In [13]:
weights, train_losses, val_losses = batch_gradient_descent(
    X_train, y_train, X_test, y_test,
    learning_rate=0.1,
    lambda_reg=0.01,
    epochs=1000,
    batch_size=32,
    patience=20
)

y_train_pred = predict(X_train, weights)
y_val_pred = predict(X_test, weights)

train_acc = accuracy(y_train, y_train_pred)
val_acc = accuracy(y_test, y_val_pred)

print(f"训练集准确率: {train_acc:.4f}")
print(f"验证集准确率: {val_acc:.4f}")


早停在第 926 轮
训练集准确率: 0.9583
验证集准确率: 0.9667
